In [ ]:
## main
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from imblearn.over_sampling import SMOTE
from PIL import Image

## skelarn -- preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline, FeatureUnion

## skelarn -- models
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

## sklearn -- metrics
from sklearn.metrics import f1_score, confusion_matrix


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
TRAIN_PATH = os.path.join(os.getcwd(), "dataset.csv")
df = pd.read_csv(TRAIN_PATH)
df.head()

In [ ]:
df.drop(columns=["CustomerId" , "RowNumber" , "Surname"], inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum() # check for duplicates

In [ ]:
df.head()

In [ ]:
class OutlierHandler(BaseEstimator, TransformerMixin):
    def __init__(self, method="iqr", factor=1.5):
        self.method = method
        self.factor = factor
    
    def fit(self, X, y=None):
        
        if self.method == "iqr":
            Q1 = np.percentile(X, 25, axis=0)
            Q3 = np.percentile(X, 75, axis=0)
            IQR = Q3 - Q1
            self.lower_bound_ = Q1 - self.factor * IQR
            self.upper_bound_ = Q3 + self.factor * IQR
        return self
    
    def transform(self, X):
        X = np.where(X < self.lower_bound_, self.lower_bound_, X)
        X = np.where(X > self.upper_bound_, self.upper_bound_, X)
        return X

In [ ]:
## To features and target
X = df.drop(columns=['Exited'], axis=1)
y = df['Exited']

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42 , stratify=y)

In [ ]:
categ_cols = df.select_dtypes(include=['object']).columns.tolist()
num_cols = ['Age', 'CreditScore', 'Tenure', 'Balance' , 'EstimatedSalary'] 
ready_cols = ['NumOfProducts' ,'HasCrCard' ,'IsActiveMember']

In [ ]:
categ_cols

In [ ]:
num_pipeline = Pipeline(steps = [
    ('imputer' , SimpleImputer(strategy='median')),
    ('outlier_handler' , OutlierHandler(method='iqr', factor=1.5)),
    ('scaler' , RobustScaler())
])

categ_pipline = Pipeline(steps=[
    ('imputer' , SimpleImputer(strategy='most_frequent')),
    ("ohe" , OneHotEncoder(handle_unknown='ignore' , drop='first'))
])

ready_pipeline = Pipeline(steps=[
    ('imputer' , SimpleImputer(strategy='most_frequent')),
])

from sklearn.compose import ColumnTransformer
all_pipeline = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_cols),
    ('cat', categ_pipline, categ_cols),
    ('ready', ready_pipeline, ready_cols)
])

In [ ]:
## apply
X_train_final = all_pipeline.fit_transform(X_train)
X_test_final = all_pipeline.transform(X_test)

In [ ]:
# Fix class weight calculation
vals_count = np.bincount(y_train) / len(y_train)

# Create balanced class weights (inverse of frequency)
dict_wieght = {}
for i in range(2):
    dict_wieght[i] = 1.0 / vals_count[i] if vals_count[i] > 0 else 1.0

# Normalize weights
total_weight = sum(dict_wieght.values())
for i in range(2):
    dict_wieght[i] = dict_wieght[i] / total_weight

## 3. Using SMOTE for over sampling
over = SMOTE(sampling_strategy=0.7)
X_train_resmapled, y_train_resampled = over.fit_resample(X_train_final, y_train)


In [ ]:
def train_model(X_train , y_train , plot_name = '', class_weight = None ):
    global clf_name
    clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight)
    clf.fit(X_train , y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test_final)
    f1_score_train = f1_score(y_train , y_pred_train)
    f1_score_test = f1_score(y_test , y_pred_test)
    clf_name = clf.__class__.__name__
    ## Plot the confusion matrix 
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, cbar=False, fmt='.2f', cmap='Blues')
    plt.title(f'{plot_name}')
    plt.xticks(ticks=np.arange(2) + 0.5, labels=[False, True])
    plt.yticks(ticks=np.arange(2) + 0.5, labels=[False, True])
    ## Save the plot locally
    plt.savefig(f'{plot_name}.png', bbox_inches='tight', dpi=300)
    plt.close()
    ## Write scores to a file
    with open('metrics.txt', 'a') as f:
        f.write(f'{clf_name} {plot_name}\n')
        f.write(f"F1-score of Training is: {f1_score_train*100:.2f} %\n")
        f.write(f"F1-Score of Validation is: {f1_score_test*100:.2f} %\n")
        f.write('----'*10 + '\n')
    return True

In [ ]:
## 1- train model without use imblancing handle 
train_model(X_train_final , y_train , plot_name = 'Without Imbalance Handle' , class_weight = None)

## 2 - train model with use imblancing handle (class_weight)
train_model(X_train_final , y_train , plot_name = 'With Imbalance Handle (class_weight)' , class_weight = dict_wieght)

## 3- train model with use imblancing handle (SMOTE)
train_model(X_train_resmapled , y_train_resampled , plot_name = 'With Imbalance Handle (SMOTE)' , class_weight = None)


In [ ]:
## Combine all conf matrix in one
confusion_matrix_paths = ['./Without Imbalance Handle.png', './With Imbalance Handle (class_weight).png', './With Imbalance Handle (SMOTE).png']

## Load and plot each confusion matrix
plt.figure(figsize=(15, 5))  # Adjust figure size as needed
for i, path in enumerate(confusion_matrix_paths, 1):
    img = Image.open(path)
    plt.subplot(1, len(confusion_matrix_paths), i)
    plt.imshow(img)
    plt.axis('off')  # Disable axis for cleaner visualization


## Save combined plot locally
plt.suptitle(clf_name, fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(f'conf_matrix.png', bbox_inches='tight', dpi=300)

## Delete old image files
for path in confusion_matrix_paths:
    os.remove(path)